In [1]:
import os # miscellaneous operating system interfaces
import shutil # high-level file operations
import numpy as np
import cv2
import tensorflow as tf
import keras
import random
from itertools import product
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Conv2D 
from tensorflow.keras.layers import MaxPooling2D 
from tensorflow.keras.layers import Flatten 
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
#from keras.applications import MobileNetV2, Xception, DenseNet121,ResNet50V2,ResNet101V2, ResNet152V2,NASNetLarge,InceptionV3,InceptionResNetV2
from tensorflow.keras.applications import ResNet50V2,MobileNetV2,ResNet152V2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import  Input, Conv2D, Conv2DTranspose, ReLU,AveragePooling2D, MaxPooling2D, Dense, BatchNormalization, GlobalAveragePooling2D, Concatenate
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import backend as K

import matplotlib.pyplot as plt




base_dir = "/content/drive/My Drive/Colab Notebooks/datasets/shoes_classifier"

train_img_dir = "/content/drive/My Drive/Colab Notebooks/datasets/shoes_classifier/train"

test_img_dir = "/content/drive/My Drive/Colab Notebooks/datasets/shoes_classifier/test"

val_img_dir = "/content/drive/My Drive/Colab Notebooks/datasets/shoes_classifier/val"


learning_rate = 0.00001
train_batch_size = 8 # train data 숫자
test_batch_size = 8 # test data 숫자
val_batch_size = 8 # val data 숫자



train_datagen = ImageDataGenerator(rescale=1./255,
                                 rotation_range = 30,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1,
                                 shear_range=0.05,
                                 zoom_range=0.05,
                                 horizontal_flip=True,
                                 fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)




categories = ['nike_coat_legacy'              
              ,'adidas_equipment10'
              ,'converse_chuckTailor_allstar_classic'
              ,'fila_coatDelux_belcro'
              ,'puma_kaia_platform'
              ,'discovery_jogger_flex'
              ,'reebok_royal_bridge3'
              ,'newbal_w480'
              ,'adidas_superstar'
              ,'nike_revolution5'
              ,'converse_chuckTailor_dainti_mule'
              ,'fila_classic_border_stitch'
              ,'puma_smash_bulk_mule'
              ,'discovery_bucket_dewalker_v2'
              ,'reebok_zig_dynamica'
              ,'newbal_ms237'
              ,'nike_air_tailwind79'
              ,'adidas_responseSR'
              ,'fila_como_mule'
              ,'puma_thunder_passion2.0'
              ,'converse_chuckTailor_move_high'
              ,'discovery_bucket_mountain_LT'
              ,'reebok_energen_run'
              ,'newbal_mr530'
              ,'nike_airmax_infinity2'
              ,'adidas_brabada'
              ,'fila_classic_kicksB'
              ,'puma_future_rider_play_on'
              ,'converse_chuckTailor_allstar_high'
              ,'discovery_brick'
              ,'reebok_royal_run_finish'
              ,'newbal_sd5205'
              ,'nike_coat_legacy_mule'
              ,'adidas_retro_bulk'
              ,'fila_bumper_mule'
              ,'puma_bari_mule'
              ,'prospecs_bigstar101'
              ,'newbal_mt410'
              ,'fila_aztrack96'
              ,'converse_chuckTailor_allStar'
              ,'puma_interplex_runner'
              ,'fila_bumper'
              ,'adidas_grand_coat_base'
              ,'nike_airmax_vg-r'
              ,'prospecs_maharun402'
              ,'newbal_ml860'
              ,'reebok_royal_run_finish2'
              ,'asics_jog_100s'
              ,'nike_airmax_bolt'
              ,'adidas_alpha_bounce_ek'
              ,'fila_rgb_flow'
              ,'puma_bari_z'
              ,'asics_jel1090'
              ,'prospecs_crossboa'
              ,'newbal_ml827'
              ,'reebok_liquifect_90ap'
              ,'nike_wear_allday'
              ,'adidas_flood_flow'
              ,'fila_classic_kicks'
              ,'puma_extra'
              ,'reebok_driftium'
              ,'newbal_ws109'
              ,'prospecs_river_ex511'
              ,'asics_jel-contend7'
              ,'nike_air_monarch4'
              ,'adidas_duramo_sl'
              ,'fila_fluid'
              ,'puma_cel_endura_patent98'
              ,'reebok_royal_complete3_low'
              ,'newbal_mflsht'
              ,'prospecs_river_ex512'
              ,'asics_trabuco_max'
              ,'nike_airmax_axi'
              ,'adidas_nizza_trefoli'
              ,'fila_decypher'
              ,'puma_tx-3'
              ,'reebok_ridgerider5.0_uni'
              ,'newbal_m068'
              ,'prospecs_stax_tr'
              ,'nike_run_swift2'
              ,'adidas_galaxy5'
              ,'fila_festibo91'
              ,'puma_scotch_runner'
              ,'prospecs_cross_tr'
              ,'newbal_wx452'
              ,'reebok_furylight3'
              ,'nike_airmax_sc'
              ,'adidas_run_palcon2.0'
              ,'fila_funky_tennis1998'
              ,'puma_millenio'
              ,'prospecs_daytrip'
              ,'newbal_yacstr'
              ,'reebok_royal_turbo_impulse'
              ,'puma_r78_og'
              ,'fila_skipper'
              ,'fila_ray_tracer'
              ,'adidas_hiwi'
              ,'adidas_clima_warm2.0u'
              ,'nike_coat_vision_alta_txt'
              ,'nike_down_shifter11'
              ]

training_set = train_datagen.flow_from_directory(train_img_dir, target_size=(300,300), 
                                             classes=categories, 
                                             batch_size=train_batch_size,
                                             class_mode='categorical'
                                             )

test_set = test_datagen.flow_from_directory(test_img_dir,
                                        target_size=(300,300), 
                                        classes=categories, 
                                        batch_size=test_batch_size,
                                        class_mode='categorical')

val_set = test_datagen.flow_from_directory(val_img_dir,
                                        target_size=(300,300), 
                                        classes=categories, 
                                        batch_size=val_batch_size,
                                        class_mode='categorical')



x_train, y_train = next(training_set)
x_test, y_test = next(test_set)
x_val,y_val = next(val_set)




Found 1129 images belonging to 100 classes.
Found 322 images belonging to 100 classes.
Found 324 images belonging to 100 classes.


In [2]:
def create_model():
    inputs = Input(shape=(300, 300, 3))    
    application  = ResNet152V2(weights = "imagenet", include_top=False,input_shape = (300,300,3)
                            ,input_tensor = inputs)
    

    output = application.output
    pooling = AveragePooling2D(pool_size=(16,16),padding='SAME')(output)   

    flatten1 = Flatten()(pooling)

    dense1 = Dense(units = 256)(flatten1)
    batch1 = BatchNormalization()(dense1)
    relu1 = ReLU()(batch1)
    dense2 = Dense(units = 64)(relu1)
    batch2 = BatchNormalization()(dense2)
    relu2 = ReLU()(batch2)

    dense3 = Dense(units = 100, activation = 'softmax')(relu2)    
    return tf.keras.Model(inputs=inputs, outputs=dense3)
    
def mobile_net():
    inputs = Input(shape=(128, 128, 3))
    mobileNet = MobileNetV2(weights = "imagenet", include_top=False,input_shape = (128,128,3)
                            ,input_tensor = inputs)
    for layer in mobileNet.layers:
        layer.trainable = True
        
    output = mobileNet.output
    pooling = AveragePooling2D(pool_size=(16,16),padding='SAME')(output)
    flatten1 = Flatten()(pooling)
    dense1 = Dense(units = 256)(flatten1)
    batch1 = BatchNormalization()(dense1)
    relu1 = ReLU()(batch1)
    dense2 = Dense(units = 64)(relu1)
    batch2 = BatchNormalization()(dense2)
    relu2 = ReLU()(batch2)

    dense3 = Dense(units = 100, activation = 'softmax')(relu2)    
    return tf.keras.Model(inputs=inputs, outputs=dense3)

def xception():
    inputs = Input(shape=(128, 128, 3))
    xception = Xception(weights = "imagenet", include_top=False,input_shape = (128,128,3)
                            ,input_tensor = inputs)
    for layer in xception.layers:
        layer.trainable = True
        
    output = xception.output
    pooling = AveragePooling2D(pool_size=(8,8),padding='SAME')(output)
    flatten1 = Flatten()(pooling)
    dense1 = Dense(units = 256)(flatten1)
    batch1 = BatchNormalization()(dense1)
    relu1 = ReLU()(batch1)
    dense2 = Dense(units = 64)(relu1)
    batch2 = BatchNormalization()(dense2)
    relu2 = ReLU()(batch2)

    dense3 = Dense(units = 100, activation = 'softmax')(relu2)    
    return tf.keras.Model(inputs=inputs, outputs=dense3)


def resnet():
    inputs = Input(shape=(128, 128, 3))
    resnet = ResNet152V2 (weights = "imagenet", include_top=False,input_shape = (128,128,3)
                            ,input_tensor = inputs)
    for layer in resnet.layers:
        layer.trainable = True
        
    output = resnet.output
    pooling = MaxPooling2D(pool_size=(8,8),padding='SAME')(output)
    flatten1 = Flatten()(pooling)
    dense1 = Dense(units = 256)(flatten1)
    batch1 = BatchNormalization()(dense1)
    relu1 = ReLU()(batch1)
    dense2 = Dense(units = 64)(relu1)
    batch2 = BatchNormalization()(dense2)
    relu2 = ReLU()(batch2)

    dense3 = Dense(units = 100, activation = 'softmax')(relu2)    
    return tf.keras.Model(inputs=inputs, outputs=dense3)

def densenet():
    inputs = Input(shape=(128, 128, 3))
    densenet = DenseNet121(weights = "imagenet", include_top=False,input_shape = (128,128,3)
                            ,input_tensor = inputs)
    for layer in densenet.layers:
        layer.trainable = True
        
    output = densenet.output
    pooling = MaxPooling2D(pool_size=(32,32),padding='SAME')(output)
    flatten1 = Flatten()(pooling)
    dense1 = Dense(units = 256)(flatten1)
    batch1 = BatchNormalization()(dense1)
    relu1 = ReLU()(batch1)
    dense2 = Dense(units = 64)(relu1)
    batch2 = BatchNormalization()(dense2)
    relu2 = ReLU()(batch2)

    dense3 = Dense(units = 100, activation = 'softmax')(relu2)    
    return tf.keras.Model(inputs=inputs, outputs=dense3)


def inception():
    inputs = Input(shape=(128, 128, 3))
    inception = InceptionV3(weights = "imagenet", include_top=False,input_shape = (128,128,3)
                            ,input_tensor = inputs)
    for layer in inception.layers:
        layer.trainable = True
        
    output = inception.output
    pooling = MaxPooling2D(pool_size=(16,16),padding='SAME')(output)
    flatten1 = Flatten()(pooling)
    dense1 = Dense(units = 256)(flatten1)
    batch1 = BatchNormalization()(dense1)
    relu1 = ReLU()(batch1)
    dense2 = Dense(units = 64)(relu1)
    batch2 = BatchNormalization()(dense2)
    relu2 = ReLU()(batch2)

    dense3 = Dense(units = 100, activation = 'softmax')(relu2)    
    return tf.keras.Model(inputs=inputs, outputs=dense3)

def inceptionresnet():
    inputs = Input(shape=(128, 128, 3))
    inceptionresnet = InceptionResNetV2(weights = "imagenet", include_top=False,input_shape = (128,128,3)
                            ,input_tensor = inputs)
    for layer in inceptionresnet.layers:
        layer.trainable = True
        
    output = inceptionresnet.output
    pooling = MaxPooling2D(pool_size=(32,32),padding='SAME')(output)
    flatten1 = Flatten()(pooling)
    dense1 = Dense(units = 256)(flatten1)
    batch1 = BatchNormalization()(dense1)
    relu1 = ReLU()(batch1)
    dense2 = Dense(units = 64)(relu1)
    batch2 = BatchNormalization()(dense2)
    relu2 = ReLU()(batch2)

    dense3 = Dense(units = 100, activation = 'softmax')(relu2)    
    return tf.keras.Model(inputs=inputs, outputs=dense3)




class ResidualUnit(tf.keras.Model):
    def __init__(self, filter_in, filter_out, kernel_size):
        super(ResidualUnit, self).__init__()
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv1 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(filter_out, kernel_size, padding='same')
        
        if filter_in == filter_out:
            self.identity = lambda x: x
        else:
            self.identity = tf.keras.layers.Conv2D(filter_out, (1,1), padding='same')

    def call(self, x, training=False, mask=None):
        h = self.bn1(x, training=training)
        h = tf.nn.relu(h)
        h = self.conv1(h)
        
        h = self.bn2(h, training=training)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        return self.identity(x) + h
    
class ResnetLayer(tf.keras.Model):
    def __init__(self, filter_in, filters, kernel_size):
        super(ResnetLayer, self).__init__()
        self.sequence = list()
        for f_in, f_out in zip([filter_in] + list(filters), filters):
            self.sequence.append(ResidualUnit(f_in, f_out, kernel_size))

    def call(self, x, training=False, mask=None):
        for unit in self.sequence:
            x = unit(x, training=training)
        return x
    
class ResNet(tf.keras.Model):
    def __init__(self):
        super(ResNet, self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu') # 28x28x8
        
        self.res1 = ResnetLayer(64, (16, 16), (3, 3)) # 28x28x16
        self.pool1 = tf.keras.layers.MaxPool2D((2, 2)) # 14x14x16
        
        
        self.res2 = ResnetLayer(128, (32, 32), (3, 3)) # 14x14x32
        self.pool2 = tf.keras.layers.MaxPool2D((2, 2)) # 7x7x32
    
        
        self.res3 = ResnetLayer(256, (64, 64), (3, 3)) # 7x7x64
        self.pool = tf.keras.layers.MaxPool2D((2, 2)) # 7x7x32
        
        
        self.res4 = ResnetLayer(512, (64, 64), (3, 3)) # 7x7x64
        self.pool = tf.keras.layers.MaxPool2D((2, 2)) # 7x7x32
        
        
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(1024, activation='relu')
        self.dense2 = tf.keras.layers.Dense(5, activation='softmax')
        
    def call(self, x, training=False, mask=None):
        x = self.conv1(x)
        
        x = self.res1(x, training=training)
        x = self.pool1(x)
        x = self.res2(x, training=training)
        x = self.pool2(x)
        x = self.res3(x, training=training)
        
        x = self.flatten(x)
        x = self.dense1(x)
        return self.dense2(x)

In [3]:
def recall(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_target_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall


def precision(y_target, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
    y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_target_yn * y_pred_yn) 

    # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
    count_true_positive_false_positive = K.sum(y_pred_yn)

    # Precision = (True Positive) / (True Positive + False Positive)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

    # return a single tensor value
    return precision


def f1score(y_target, y_pred):
    _recall = recall(y_target, y_pred)
    _precision = precision(y_target, y_pred)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
    # return a single tensor value
    return _f1score



model = create_model()
#model = ResNet()
#model = mobile_net()
#model = xception()
#model = densenet()
#model = resnet()
#model = inception()
#model =inceptionresnet()


lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=100000,
    decay_rate=0.96)

model.compile(keras.optimizers.Adam(lr_schedule), loss = 'categorical_crossentropy', metrics=['accuracy', precision, recall, f1score])



history = model.fit(
    training_set, 
    epochs=120,
    validation_data=val_set,
)


Epoch 1/120


KeyboardInterrupt: ignored

In [ ]:
model.save('shoes_classifier.h5')

In [ ]:
# output = classifier.predict_generator(test_set, steps=1)
# print(test_set.class_indices)
# print(output)
size = y_test[:,-1]

filenames = test_set.filenames
nb_samples = len(filenames) / test_batch_size


# predict 10 random hand-writing data
y_predicted = model.predict(test_set, steps=nb_samples)

for x in range(0,size.size):
    
   np.set_printoptions(suppress=True)
   print("index:", x,
          " actual y:", np.argmax(y_test[x]),
          " answer y:", np.argmax(y_predicted[x]),
          " prediction:", np.array(y_predicted[x] * 100))

_loss, _acc, _precision, _recall, _f1score = model.evaluate(test_set, steps=nb_samples)
print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _precision, _recall, _f1score))
# print('loss: ', evaluation[0])
# print('accuracy', evaluation[1])

import matplotlib.pyplot as plt

# 손실 그래프
def plot_loss(history):
   # 선 그리기
    plt.plot(history.history['loss'], 'y', label='train loss')
    plt.plot(history.history['val_loss'], 'r', label='val loss')
   # 그래프 제목
    plt.title('Model Loss')
   # x,y축 이름 표시
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
   # 각 라인 표식 표시
    plt.legend(['Train','Validation'],loc=0)

# 정확도 그래프
def plot_acc(history):
  # dir(history.history)
    plt.plot(history.history['accuracy'], 'b', label='train accuracy')
    plt.plot(history.history['val_accuracy'], 'g', label='val accuracy')
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc=0)

plot_loss(history)
plt.show()
plot_acc(history)
plt.show()

In [ ]:
# parameter tuning시 사용하는 code


# batch_size = [8,16,32,64,128,256]
# learning_rate = [0.000001,0.00001,0.0001,0.001,0.01]

# # batch_size = [16]
# # learning_rate = [0.001]



# def recall(y_target, y_pred):
#     # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
#     # round : 반올림한다
#     y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
#     y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

#     # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
#     count_true_positive = K.sum(y_target_yn * y_pred_yn) 

#     # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
#     count_true_positive_false_negative = K.sum(y_target_yn)

#     # Recall =  (True Positive) / (True Positive + False Negative)
#     # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
#     recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

#     # return a single tensor value
#     return recall


# def precision(y_target, y_pred):
#     # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
#     # round : 반올림한다
#     y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다
#     y_target_yn = K.round(K.clip(y_target, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다

#     # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
#     count_true_positive = K.sum(y_target_yn * y_pred_yn) 

#     # (True Positive + False Positive) = 예측 값이 1(Positive) 전체
#     count_true_positive_false_positive = K.sum(y_pred_yn)

#     # Precision = (True Positive) / (True Positive + False Positive)
#     # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
#     precision = count_true_positive / (count_true_positive_false_positive + K.epsilon())

#     # return a single tensor value
#     return precision


# def f1score(y_target, y_pred):
#     _recall = recall(y_target, y_pred)
#     _precision = precision(y_target, y_pred)
#     # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
#     _f1score = ( 2 * _recall * _precision) / (_recall + _precision+ K.epsilon())
    
#     # return a single tensor value
#     return _f1score

# for  batch_size, learning_rate in product(batch_size,learning_rate):
    
#     # Part 2 - Fitting the CNN to the images
#     print(f'batchSIZE is {batch_size}, Learning Rate is {learning_rate}')
#     print(f'train_epochs is {training_epochs}, validation_step is {validation_steps}')
#     # train_datagen = ImageDataGenerator(rescale = 1./255,
#     #                                    horizontal_flip = True)
#     # val_datagen = ImageDataGenerator(rescale = 1./255)

   
#     # test_datagen = ImageDataGenerator(rescale = 1./255)

#     categories = ['nike_coat_legacy',
#               'nike_revolution5',
#               'adidas_equipment10'
#               ,'converse_chuckTailor_allStarClassic_leather']

#     # training_set = train_datagen.flow_from_directory(train_img_dir, target_size=(128,128), 
#     #                                                  classes=categories, 
#     #                                                  batch_size=batch_size)

#     # test_set = test_datagen.flow_from_directory(test_img_dir,
#     #                                             target_size=(128,128), 
#     #                                             classes=categories, 
#     #                                             batch_size=batch_size)
    
#     # val_set = test_datagen.flow_from_directory(val_img_dir,
#     #                                     target_size=(128,128), 
#     #                                     classes=categories, 
#     #                                     batch_size=batch_size)



#     # x_train, y_train = next(training_set)
#     # x_test, y_test = next(test_set)
#     # x_val,y_val = next(val_set)

#     def create_model():
#       inputs = Input(shape=(299, 299, 3))
#       inceptionresnet = InceptionResNetV2(weights = "imagenet", include_top=False,input_shape = (299,299,3)
#                               ,input_tensor = inputs)
#       for layer in inceptionresnet.layers:
#           layer.trainable = True
          
#       output = inceptionresnet.output
#       pooling = MaxPooling2D(pool_size=(32,32),padding='SAME')(output)
#       flatten1 = Flatten()(pooling)
#       dense1 = Dense(units = 1024)(flatten1)
#       batch1 = BatchNormalization()(dense1)
#       relu1 = ReLU()(batch1)
#       dense2 = Dense(units = 512)(relu1)
#       batch2 = BatchNormalization()(dense2)
#       relu2 = ReLU()(batch2)
#       dense3 = Dense(units = 4, activation = 'softmax')(relu2)
#       return keras.Model(inputs=inputs, outputs=dense3) 
      
#     model = create_model()

#     model.compile(
#     optimizer=keras.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999),
#     loss='categorical_crossentropy', 
#     metrics=['accuracy', precision, recall, f1score],
#     )

    

#     history = model.fit(
#     training_set, 
#     epochs=90,
#     validation_data=val_set,
#     )


#     size = y_test[:,-1]
#     print(y_test.size)
#     print(size.size)


#     # predict 10 random hand-writing data
#     y_predicted = model.predict(x_test)
#     for x in range(0,size.size):
        
#       np.set_printoptions(suppress=True)
#       print("index:", x,
#               " actual y:", np.argmax(y_test[x]),
#               " answer y:", np.argmax(y_predicted[x]),
#               " prediction:", np.array(y_predicted[x] * 100))

#     _loss, _acc, _precision, _recall, _f1score = model.evaluate(x_test, y_test)
#     print('loss: {:.3f}, accuracy: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _precision, _recall, _f1score))
#     # print('loss: ', evaluation[0])
#     # print('accuracy', evaluation[1])



#     # 손실 그래프
#     def plot_loss(history):
#       # 선 그리기
#         plt.plot(history.history['loss'], 'y', label='train loss')
#         plt.plot(history.history['val_loss'], 'r', label='val loss')
#       # 그래프 제목
#         plt.title('Model Loss')
#       # x,y축 이름 표시
#         plt.ylabel('Loss')
#         plt.xlabel('Epoch')
#       # 각 라인 표식 표시
#         plt.legend(['Train','Validation'],loc=0)

#     # 정확도 그래프
#     def plot_acc(history):
#       # dir(history.history)
#         plt.plot(history.history['accuracy'], 'b', label='train accuracy')
#         plt.plot(history.history['val_accuracy'], 'g', label='val accuracy')
#         plt.title('Model accuracy')
#         plt.ylabel('Accuracy')
#         plt.xlabel('Epoch')
#         plt.legend(['Train', 'Validation'], loc=0)

#     plot_loss(history)
#     plt.show()
#     plot_acc(history)
#     plt.show()